# Project

# 1.1 Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import copy
import sklearn
import sklearn.preprocessing
import sklearn.neural_network

np.set_printoptions(precision=3, suppress=True)  # Print as 0.001 instead of 9.876e-4
torch.set_printoptions(precision=3, sci_mode=False)
!pip install git+https://github.com/epistimio/orion.git@develop
!pip install orion[profet]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/epistimio/orion.git (to revision develop) to /tmp/pip-req-build-l79gg7dg
  Running command git clone --filter=blob:none --quiet https://github.com/epistimio/orion.git /tmp/pip-req-build-l79gg7dg
  Resolved https://github.com/epistimio/orion.git to commit 3a4b49f1f959193520efa15b894d41f19864aa3c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 108.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 15.8 MB/s eta 0:00:00
     ━━━━

# 1.2 User Defined Functions

In [ ]:
def plot_matrix_grid(V, cmap='bwr'):
    """
    Given an array V containing stacked matrices, plots them in a grid layout.
    V should have shape (K,M,N) where V[k] is a matrix of shape (M,N).
    The default cmap is "bwr" (blue-white-red) but can also be "gray".
    """
    if isinstance(V, torch.Tensor):
        V = V.detach().numpy()
    assert V.ndim == 3, "Expected V to have 3 dimensions, not %d" % V.ndim
    k, m, n = V.shape
    ncol = 8                                     # At most 8 columns
    nrow = min(4, (k + ncol - 1) // ncol)        # At most 4 rows
    V = V[:nrow*ncol]                            # Focus on just the matrices we'll actually plot
    figsize = (2*ncol, max(1, 2*nrow*(m/n)))     # Guess a good figure shape based on ncol, nrow
    fig, axes = plt.subplots(nrow, ncol, sharex=True, sharey=True, figsize=figsize)
    vmax = np.percentile(np.abs(V), [99.9])      # Show the main range of values, between 0.1%-99.9%
    for v, ax in zip(V, axes.flat):
        img = ax.matshow(v, vmin=-vmax, vmax=vmax, cmap=plt.get_cmap(cmap))
        ax.set_xticks([])
        ax.set_yticks([])
    for ax in axes.flat[len(V):]:
        ax.set_axis_off()
    fig.colorbar(img, cax=fig.add_axes([0.92, 0.25, 0.01, .5]))   # Add a colorbar on the right    

In [ ]:
def plot_named_tensors(tensor_dict):
    """
    Given a dict of {name: tensor} pairs, plots the tensors side-by-side in a common
    color scale. The name of each tensor is shown above its plot.
    """
    n = len(tensor_dict)
    vmax = max(v.abs().max() for v in tensor_dict.values())
    figsize = (2*n, 6)
    fig, axes = plt.subplots(1, n, figsize=figsize,  constrained_layout=True, squeeze=True)
    axes = axes.flat if isinstance(axes, np.ndarray) else (axes,)
    for (name, v), ax in zip(tensor_dict.items(), axes):
        v = torch.squeeze(v.detach())   # Automatically convert (N,1,D) to (N,D)
        if v.ndim == 1:
            v = v.view(-1, 1)  # Automatically convert (N,) to (N,1)
        assert v.ndim == 2, "couldn't turn tensors[%d] with shape %s into 2D" % (i, v.shape)
        img = ax.matshow(v, vmin=-vmax, vmax=vmax, cmap=plt.get_cmap('bwr'))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(name)
    fig.colorbar(img, cax=fig.add_axes([0.985, 0.25, 0.03, .5]))   # Add a colorbar on the right    

In [ ]:
# From Tutorial from week 5 (Debug)
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
torch.manual_seed(1)

# Hparams
# batch_size=128
# epochs=20 
# lr=0.005
# input_size = 784
# hidden_size = 500
num_classes = 10 

device = "cpu"
if torch.cuda.is_available():
  device = "cuda"


def train(X_trn, y_trn, model, loss, args, optimizer):
  losses=[]
  accuracies=[]
  model.train()

  for i in range(0, len(X_trn), args.batchsize):
    images = X_trn[i:i+args.batchsize].to(device)
    labels = y_trn[i:i+args.batchsize].to(device)
    #print('images.shape',images.shape)
    outputs = model(images)
    l = loss(outputs, labels)
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(l.item())
    
    # Append Accuracy
    predictions = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    accuracies.append(sum(predictions == labels).item()/args.batchsize)
  return torch.tensor(losses).mean(), torch.tensor(accuracies).mean()

def test(X_tst, y_tst, model, loss, args):
  losses=[]
  accuracies=[]
  err=0
  model.eval()
  for i in range(0, len(X_tst), args.batchsize):
    images = X_tst[i:i+args.batchsize].to(device)
    labels = y_tst[i:i+args.batchsize].to(device)
    outputs = model(images)
    l = loss(outputs, labels)
    losses.append(l.item())

    predictions = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    accuracies.append(sum(predictions == labels).item()/args.batchsize)
    err = err + (predictions!=labels).sum().item()
  return torch.tensor(losses).mean(), err/len(X_tst), torch.tensor(accuracies).mean()

# Training loop
def plot_epochs(X_trn, y_trn, X_tst, y_tst, model, loss, args, plot=False):
  train_losses=[]
  test_losses=[]
  train_accuracies=[]
  test_accuracies=[]
  optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weightdecay)

  for epoch in range(args.epochs):
    train_loss, train_accuracy = train(X_trn, y_trn, model, loss, args, optimizer)
    test_loss, test_err, test_accuracy = test(X_tst, y_tst, model, loss, args)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    print('Epoch: {}  train_loss={:.4f}, test_loss={:.4f}, test_err={:.2f}%, train_accuracy={:.2f}%, test_accuracy={:.2f}%' \
      .format(epoch+1, train_loss, test_loss, test_err*100, train_accuracy*100, test_accuracy*100))

  if plot == True:
    plt.figure(figsize=(10, 5))
    plt.subplot(1,2,1)
    plt.plot(train_losses, '-s', label='train')
    plt.plot(test_losses, '-s', label='test')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(train_accuracies, '-s', label='train')
    plt.plot(test_accuracies, '-s', label='test')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

  return train_accuracies[-1], test_accuracies[-1]

# Preprocess Image
# From https://huggingface.co/spaces/pytorch/ResNet/blob/main/app.py
def preprocess_image(input_image):
  preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image);
    
  return input_tensor;


<div style="border-bottom: 3px solid black; margin-bottom:5px"></div>
<div style="border-bottom: 3px solid black"></div>

# 2. Getting data

Data obtained from the following dataset in Kaggle:
https://www.kaggle.com/datasets/moodrammer/handdrawn-circuit-schematic-components

In [ ]:
def get_data():
  data_retrieval = 0

  if data_retrieval == 0:
    !gdown 11Psi1ZZ2WJ-oHCG1UZcyjGQ1N1iFllDf
    path_for_npz = "data.npz"

  else:
    colab_notebook_root = "/content/drive/MyDrive/Colab Notebooks"
    path_for_npz = colab_notebook_root + "/COMP 432/Project/npz/data.npz"
    if data_retrieval == 1:
        path_for_npz = "G:/My Drive/Colab Notebooks/COMP 432/Project/npz/data.npz"
    else:
        from google.colab import drive
        drive.mount("/content/drive")

  #!gdown 11Psi1ZZ2WJ-oHCG1UZcyjGQ1N1iFllDf
  #!rm data.npz
  #!find data.npz

  with np.load(path_for_npz) as data:
    X = data['X']
    print(X.shape)
    X = X[:,:,:,2] # Get one channel
    print('X.shape', X.shape)
    X = X.reshape(-1, 1, 120, 120)
    X = X / 255.0 #Normalize data

    y = data['y']
    print('y.shape', y.shape)
    # Encode labels to integers
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    le.fit(y)
    print('label classes', le.classes_.shape)
    y = le.transform(y)

    return X, y

X, y = get_data()

In [ ]:
print('X.shape',X.shape)
print('y.shape', y.shape)
# Your code for printing sample values. Aim for 2 lines.
print(X[0][400:415])
# Your code for printing sample values. Aim for 2 lines.
print(y[1:5])
#print(X_trn)
print('Min X', np.min(X))
print('Max X', np.max(X))

In [ ]:
# Split train and test data (aim for 1 line)
def get_and_split_data(train_size):
  X, y = get_data()
  X_trn, X_tst, y_trn, y_tst = sklearn.model_selection.train_test_split(X, y, train_size=train_size, random_state=0) 
  X_trn = torch.from_numpy(X_trn)
  X_tst = torch.from_numpy(X_tst)
  y_trn = torch.from_numpy(y_trn)
  y_tst = torch.from_numpy(y_tst)
  return X_trn, X_tst, y_trn, y_tst

X_trn, X_tst, y_trn, y_tst = get_and_split_data(0.70)
print(X.shape)
print(y.shape)
print(X_trn.shape)
print(X_tst.shape)














# 2.1 Plain CNN

**Complete the code below**

In [ ]:
%%file plain_cnn.py

torch.manual_seed(0) # Ensure model weights initialized with same random numbers

# Your code here. Aim for 8-11 lines.
drop_out_value = 0

plain_cnn_model = torch.nn.Sequential( #120 X 120 X 1
    torch.nn.Conv2d(1, 8, kernel_size=5, padding=2), #120 X 120 X ?
    # torch.nn.Dropout(drop_out_value),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2), #60 X 60 X ?

    torch.nn.Conv2d(8, 16, kernel_size=5, padding=2), #60 X 60 X ?
    # torch.nn.Dropout(drop_out_value),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2), #30 X 30 X ?

    torch.nn.Conv2d(16, 32, kernel_size=5, padding=2), #30 X 30 X ?
    # torch.nn.Dropout(drop_out_value),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2), #15 X 15 X ?

    torch.nn.Conv2d(32, 64, kernel_size=5, padding=2), #15 X 15 X ?
    # torch.nn.Dropout(drop_out_value),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2), #7 X 7 X ?

    torch.nn.Conv2d(64, 128, kernel_size=5, padding=2), #15 X 15 X ?
    # torch.nn.Dropout(drop_out_value),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2), #7 X 7 X ?

    torch.nn.Flatten(1,-1),
    torch.nn.Linear(1152,15)
)

# 2.2 Hyperparameter Tuning

In [ ]:
%%file data_prep.py

import argparse
import numpy as np
import sklearn
import sklearn.preprocessing
import sklearn.neural_network
import torch
import copy
import torch.nn.functional as F
from orion.client import report_objective

def get_data():
  data_retrieval = 0

  if data_retrieval == 0:
    #!gdown 11Psi1ZZ2WJ-oHCG1UZcyjGQ1N1iFllDf
    path_for_npz = "data.npz"

  else:
    colab_notebook_root = "/content/drive/MyDrive/Colab Notebooks"
    path_for_npz = colab_notebook_root + "/COMP 432/Project/npz/data.npz"
    if data_retrieval == 1:
        path_for_npz = "G:/My Drive/Colab Notebooks/COMP 432/Project/npz/data.npz"
    else:
        from google.colab import drive
        drive.mount("/content/drive")

  with np.load(path_for_npz) as data:
    X = data['X']
    X = X[:,:,:,2] # Get one channel
    X = X.reshape(-1, 1, 120, 120)
    X = X / 255.0 #Normalize data

    y = data['y']
    # Encode labels to integers
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)

    return X, y


In [ ]:
%%file plain_cnn_model.py
import torch

def get_plain_cnn_model():
  plain_cnn_model = torch.nn.Sequential( #120 X 120 X 1
      torch.nn.Conv2d(1, 8, kernel_size=5, padding=2), #120 X 120 X ?
      # torch.nn.Dropout(drop_out_value),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2), #60 X 60 X ?

      torch.nn.Conv2d(8, 16, kernel_size=5, padding=2), #60 X 60 X ?
      # torch.nn.Dropout(drop_out_value),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2), #30 X 30 X ?

      torch.nn.Conv2d(16, 32, kernel_size=5, padding=2), #30 X 30 X ?
      # torch.nn.Dropout(drop_out_value),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2), #15 X 15 X ?

      torch.nn.Conv2d(32, 64, kernel_size=5, padding=2), #15 X 15 X ?
      # torch.nn.Dropout(drop_out_value),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2), #7 X 7 X ?

      torch.nn.Conv2d(64, 128, kernel_size=5, padding=2), #15 X 15 X ?
      # torch.nn.Dropout(drop_out_value),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=2, stride=2), #7 X 7 X ?

      torch.nn.Flatten(1,-1),
      torch.nn.Linear(1152,15)
  )

  return plain_cnn_model

In [ ]:
%%file train.py

import argparse
import numpy as np
import sklearn
import sklearn.preprocessing
import sklearn.neural_network
import torch
import copy
import torch.nn.functional as F
from orion.client import report_objective # Orion
from data_prep import get_data
from plain_cnn_model import get_plain_cnn_model

# def get_data():
#   data_retrieval = 0

#   if data_retrieval == 0:
#     #!gdown 11Psi1ZZ2WJ-oHCG1UZcyjGQ1N1iFllDf
#     path_for_npz = "data.npz"

#   else:
#     colab_notebook_root = "/content/drive/MyDrive/Colab Notebooks"
#     path_for_npz = colab_notebook_root + "/COMP 432/Project/npz/data.npz"
#     if data_retrieval == 1:
#         path_for_npz = "G:/My Drive/Colab Notebooks/COMP 432/Project/npz/data.npz"
#     else:
#         from google.colab import drive
#         drive.mount("/content/drive")

#   with np.load(path_for_npz) as data:
#     X = data['X']
#     X = X[:,:,:,2] # Get one channel
#     X = X.reshape(-1, 1, 120, 120)
#     X = X / 255.0 #Normalize data

#     y = data['y']
#     # Encode labels to integers
#     from sklearn import preprocessing
#     le = preprocessing.LabelEncoder()
#     le.fit(y)
#     y = le.transform(y)

#     return X, y

torch.manual_seed(0) # Ensure model weights initialized with same random numbers

# Your code here. Aim for 8-11 lines.
drop_out_value = 0

# plain_cnn_model = torch.nn.Sequential( #120 X 120 X 1
#     torch.nn.Conv2d(1, 8, kernel_size=5, padding=2), #120 X 120 X ?
#     # torch.nn.Dropout(drop_out_value),
#     torch.nn.ReLU(),
#     torch.nn.MaxPool2d(kernel_size=2, stride=2), #60 X 60 X ?

#     torch.nn.Conv2d(8, 16, kernel_size=5, padding=2), #60 X 60 X ?
#     # torch.nn.Dropout(drop_out_value),
#     torch.nn.ReLU(),
#     torch.nn.MaxPool2d(kernel_size=2, stride=2), #30 X 30 X ?

#     torch.nn.Conv2d(16, 32, kernel_size=5, padding=2), #30 X 30 X ?
#     # torch.nn.Dropout(drop_out_value),
#     torch.nn.ReLU(),
#     torch.nn.MaxPool2d(kernel_size=2, stride=2), #15 X 15 X ?

#     torch.nn.Conv2d(32, 64, kernel_size=5, padding=2), #15 X 15 X ?
#     # torch.nn.Dropout(drop_out_value),
#     torch.nn.ReLU(),
#     torch.nn.MaxPool2d(kernel_size=2, stride=2), #7 X 7 X ?

#     torch.nn.Conv2d(64, 128, kernel_size=5, padding=2), #15 X 15 X ?
#     # torch.nn.Dropout(drop_out_value),
#     torch.nn.ReLU(),
#     torch.nn.MaxPool2d(kernel_size=2, stride=2), #7 X 7 X ?

#     torch.nn.Flatten(1,-1),
#     torch.nn.Linear(1152,15)
# )

# Split train and test data (aim for 1 line)
def get_and_split_data(train_size):
  X, y = get_data()
  X_trn, X_tst, y_trn, y_tst = sklearn.model_selection.train_test_split(X, y, train_size=train_size, random_state=0) 
  X_val, X_tst, y_val, y_tst = sklearn.model_selection.train_test_split(X_tst, y_tst, train_size=0.5, random_state=0) 
  X_trn = torch.from_numpy(X_trn)
  X_val = torch.from_numpy(X_val)
  X_tst = torch.from_numpy(X_tst)
  y_trn = torch.from_numpy(y_trn)
  y_val = torch.from_numpy(y_val)
  y_tst = torch.from_numpy(y_tst)
  return X_trn, X_val, X_tst, y_trn, y_val, y_tst

device = "cpu"
if torch.cuda.is_available():
  device = "cuda"


def train(X_trn, y_trn, model, loss, args, optimizer):
  losses=[]
  accuracies=[]
  model.train()

  for i in range(0, len(X_trn), args.batchsize):
    images = X_trn[i:i+args.batchsize].to(device)
    labels = y_trn[i:i+args.batchsize].to(device)
    #print('images.shape',images.shape)
    outputs = model(images)
    l = loss(outputs, labels)
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(l.item())
    
    # Append Accuracy
    predictions = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    accuracies.append(sum(predictions == labels).item()/args.batchsize)
  return torch.tensor(losses).mean(), torch.tensor(accuracies).mean()

def test(X_tst, y_tst, model, loss, args):
  losses=[]
  accuracies=[]
  err=0
  model.eval()
  for i in range(0, len(X_tst), args.batchsize):
    images = X_tst[i:i+args.batchsize].to(device)
    labels = y_tst[i:i+args.batchsize].to(device)
    outputs = model(images)
    l = loss(outputs, labels)
    losses.append(l.item())

    predictions = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    accuracies.append(sum(predictions == labels).item()/args.batchsize)
    err = err + (predictions!=labels).sum().item()
  return torch.tensor(losses).mean(), err/len(X_tst), torch.tensor(accuracies).mean()

def plot_epochs(X_trn, y_trn, X_tst, y_tst, model, loss, args, plot=False):
  train_losses=[]
  test_losses=[]
  train_accuracies=[]
  test_accuracies=[]
  optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weightdecay)

  for epoch in range(args.epochs):
    train_loss, train_accuracy = train(X_trn, y_trn, model, loss, args, optimizer)
    test_loss, test_err, test_accuracy = test(X_tst, y_tst, model, loss, args)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    print('Epoch: {}  train_loss={:.4f}, test_loss={:.4f}, test_err={:.2f}%, train_accuracy={:.2f}%, test_accuracy={:.2f}%' \
      .format(epoch+1, train_loss, test_loss, test_err*100, train_accuracy*100, test_accuracy*100))

  if plot == True:
    plt.figure(figsize=(10, 5))
    plt.subplot(1,2,1)
    plt.plot(train_losses, '-s', label='train')
    plt.plot(test_losses, '-s', label='test')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(train_accuracies, '-s', label='train')
    plt.plot(test_accuracies, '-s', label='test')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

  return train_accuracies[-1], test_accuracies[-1]


def orion_train():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batchsize', type=int, default=64,
                        help='input batch size for training (default: 64)')
    parser.add_argument('--epochs', type=int, default=10, 
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=0.01, 
                        help='learning rate (default: 1.0)')
    parser.add_argument('--neurons', type=int, default=100, 
                        help='number of neurons (default: 100)')
    parser.add_argument('--eval', type=bool, default=False, 
                        help='If True it prints the test error (default: False)')
    parser.add_argument('--weightdecay', type=float, default=0, 
                        help='weight decay (default: 0)')
    parser.add_argument("-f", required=False)
    args = parser.parse_args()
    print(args)

    # Select data
    X_trn, X_val, X_tst, y_trn, y_val, y_tst = get_and_split_data(0.70)
#    model = copy.deepcopy(plain_cnn_model).to(device)
    model = copy.deepcopy(get_plain_cnn_model()).to(device)

    # Your code for defining loss, optimizer, and training loop here. Aim for 10-12 lines.
    loss = torch.nn.CrossEntropyLoss()
    train_score, val_score = plot_epochs(X_trn, y_trn, X_val, y_val, model, loss, args)
    valid_error = (100*(1 - val_score)).item()
    print("Valid Error (\%): " + str(valid_error))

    report_objective(valid_error)

    # if args.eval:
    #   test_error = 100*(1 - model.score(X_tst, y_tst))
    #   print("Test Error (\%): " + str(test_error))


if __name__ == '__main__':
    orion_train()

In [ ]:
!python train.py --lr=0.001 --epochs=3 --batchsize=64 --neurons=20 --eval='True'

Namespace(batchsize=64, epochs=3, lr=0.001, neurons=20, eval=True, weightdecay=0, f=None)
Epoch: 1  train_loss=2.4897, test_loss=1.9705, test_err=64.11%, train_accuracy=15.62%, test_accuracy=35.49%
Epoch: 2  train_loss=1.5483, test_loss=1.3213, test_err=47.63%, train_accuracy=44.74%, test_accuracy=51.79%
Epoch: 3  train_loss=1.0380, test_loss=0.9792, test_err=34.76%, train_accuracy=62.59%, test_accuracy=64.51%
Valid Error (\%): 35.49107360839844
[{'name': 'objective', 'type': 'objective', 'value': 35.49107360839844}]


In [ ]:
!orion hunt -n plain_cnn --exp-max-trials=50 python train.py  --lr~'choices([0.1,0.01,0.001,0.0001])'  --batchsize~'choices([32,64,128])' --epochs~'choices([5,10,15,20])' --weightdecay~'choices([1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0])'

Using the best results obtained from hyperparameter tuning, we try the following:

In [ ]:
!orion hunt -n plain_cnn2 --exp-max-trials=50 python train.py  --lr~'choices([0.001])'  --batchsize~'choices([32])' --epochs~'choices([25,30,35,40])' --weightdecay~'choices([0])'

In [ ]:
!python train.py --lr=0.001 --epochs=35 --batchsize=32 --weightdecay=0

Namespace(batchsize=32, epochs=35, lr=0.001, neurons=100, eval=False, weightdecay=0.0, f=None)
Epoch: 1  train_loss=2.3448, test_loss=1.7112, test_err=55.30%, train_accuracy=19.57%, test_accuracy=44.20%
Epoch: 2  train_loss=1.3838, test_loss=1.1730, test_err=41.99%, train_accuracy=50.10%, test_accuracy=57.37%
Epoch: 3  train_loss=0.8866, test_loss=0.7553, test_err=29.57%, train_accuracy=68.32%, test_accuracy=69.64%
Epoch: 4  train_loss=0.5713, test_loss=0.6767, test_err=27.09%, train_accuracy=79.18%, test_accuracy=72.10%
Epoch: 5  train_loss=0.4336, test_loss=0.6797, test_err=23.25%, train_accuracy=84.13%, test_accuracy=75.89%
Epoch: 6  train_loss=0.3175, test_loss=0.4560, test_err=17.16%, train_accuracy=87.74%, test_accuracy=81.92%
Epoch: 7  train_loss=0.2400, test_loss=0.5061, test_err=16.70%, train_accuracy=90.96%, test_accuracy=82.37%
Epoch: 8  train_loss=0.1634, test_loss=0.4455, test_err=12.87%, train_accuracy=93.46%, test_accuracy=86.16%
Epoch: 9  train_loss=0.1302, test_loss=0.

In [ ]:
!orion info --name orion-tutorial2 --version 1

Identification
name: orion-tutorial2
version: 1
user: root


Commandline
python train.py --lr~loguniform(1e-4, 0.1) --batchsize~choices([32,64,128]) --epochs~uniform(15, 50, discrete=True)


Config
max trials: 50
max broken: 3
working dir: 


Algorithm
random:
    seed: None


Space
=====
/batchsize: choices([32, 64, 128])
/epochs: uniform(15, 50, discrete=True)
/lr: loguniform(0.0001, 0.1)


Meta-data
user: root
datetime: 2023-04-10 21:47:24.609277
orion version: 0.2.6.post294+g3a4b49f1
VCS:



Parent experiment
root:
parent:
adapter:


Stats
=====
No trials executed...




# 2.3 Evaluating Plain CNN

In [ ]:
# Hyperparameters
from argparse import Namespace
args = Namespace(batchsize=64, epochs=3, lr=0.001, neurons=100, weightdecay=0, eval=False,f=False)

# # Your code to define loss function and optimizer here. Aim for 2 lines.
loss = torch.nn.CrossEntropyLoss()
# train(X_trn, y_trn, plain_cnn_model, loss, args)
# orion_train()
model = copy.deepcopy(plain_cnn_model)
plot_epochs(X_trn, y_trn, X_tst, y_tst, model.to(device), loss, args)


Epoch: 1  train_loss=2.4964, test_loss=2.0471, test_err=67.95%, train_accuracy=15.06%, test_accuracy=31.70%
Epoch: 2  train_loss=1.5477, test_loss=1.2488, test_err=44.13%, train_accuracy=44.84%, test_accuracy=55.25%
Epoch: 3  train_loss=1.0549, test_loss=0.9565, test_err=33.41%, train_accuracy=60.84%, test_accuracy=65.85%


(tensor(0.608), tensor(0.658))